## 5-6. 성능을 높이는 방법
앞서 보인 예시들도 나름 좋은 성능을 보이기는 하지만 최선을 다했다고는 볼 수 없다. 그렇다면 무엇을 더 해볼 수 있을까? 첫째는 토큰화, 정규화 등을 좀 더 세심하게 해볼 수 있다. 사이킷런의 CountVectorizer와 TfidfVectorizer는 자체 토크나이저를 쓸 수도 있지만 외부에서 정의한 함수를 사용할 수 있으므로 앞에서 배운 지식을 동원해서 토크나이저를 향상시켜본다. 아래 예시에서는 정규표현식 토크나이저를 사용하고, NLTK의 불용어 사전을 이용해 불용어를 제거했으며, 포터 스테머를 사용해서 스테밍을 수행했다. TfidfVectorizer에 이 토큰화 결과를 입력으로 사용해서 나온 특성 벡터로 로지스틱 회귀분석을 실시하여 결과를 비교해본다.

In [1]:
% store -r X_train
% store -r X_test
% store -r y_train
% store -r y_test

In [2]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

In [3]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

RegTok = RegexpTokenizer("[\w']{3,}") # 정규표현식으로 토크나이저를 정의
english_stops = set(stopwords.words('english')) # 영어 불용어를 가져옴

def tokenizer(text):
    tokens = RegTok.tokenize(text.lower())
    words = [word for word in tokens if (word not in english_stops) and len(word) > 2] # stopwords 제외
    features = (list(map(lambda token: PorterStemmer().stem(token), words))) # porter stemmer 적용
    return features

# 새로 정의한 토크나이저 사용
tfidf = TfidfVectorizer(tokenizer=tokenizer, max_features=2000, min_df=5, max_df=0.5)
X_train_tfidf = tfidf.fit_transform(X_train) # train set을 변환
X_test_tfidf = tfidf.transform(X_test) # test set을 변환

# tfidf vector를 이용해서 분류기 핛흡
LR_clf = LogisticRegression() # 분류기 선언
LR_clf.fit(X_train_tfidf, y_train) # train data를 이용해 분류기를 학습

print(f"# Train set score: {LR_clf.score(X_train_tfidf, y_train):.3f}") # train data에 대한 예측 정확도
print(f"# Test set score: {LR_clf.score(X_test_tfidf, y_test):.3f}") # test data에 대한 예측 정확도
len(LR_clf.coef_[0])

# Train set score: 0.930
# Test set score: 0.751


2000

앞서 릿지 회귀의 정확도가 0.739에 불과했던 것에 비해 0.751로 꽤 많이 향상됐다. 이 장의 시작에서 우리는 특성의 수를 2,000개로 제한했는데, 첫째 이유는 학습 데이터셋의 샘플 수가 2,034개밖에 되지 않는다는 것 때문이다. 특성의 수가 2,000개이면 그냥 회귀분석을 해도 2,000여개의 계수를 추정해야 한다. 카테고리가 4개인 로지스틱 회귀분석이라면 8,000개의 계수를 단 2,034개의 샘플로 추정해야 하는 상황이다. 일반적으로는 어마어마한 과적합 현상이 일어나서 일반화가 거의 안 돼야 하는데 문서 분류는 그 특성상 별 문제 없이 가능하다. 그렇다면 특성의 수를 더 늘려보면 어떨까? 아예 max_features를 없애버리고 모든 단어를 다 특성으로 사용한다면? 과적합 문제가 없다면 특성의 수는 많을수록 좋을 가능성이 높다.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import MultinomialNB

tfidf = TfidfVectorizer(tokenizer=tokenizer)
X_train_tfidf = tfidf.fit_transform(X_train) # train set 변환
print("# Train set dimension:", X_train_tfidf.shape) # 실제로 몇 개의 특성이 사용됐는지 확인

X_test_tfidf = tfidf.transform(X_test) # test_set을 변환
print("# Test set dimension:", X_test_tfidf.shape)

ridge_clf = RidgeClassifier(alpha=2.4)
ridge_clf.fit(X_train_tfidf, y_train) # 학습
print(f"# Train set score: {ridge_clf.score(X_train_tfidf, y_train):.3f}")
print(f"# Test set score: {ridge_clf.score(X_test_tfidf, y_test):.3f}")

NB_clf = MultinomialNB(alpha=0.01) # 분류기 선언
NB_clf.fit(X_train_tfidf, y_train) # train set을 이용해 분류기를 학습

# train set에 대한 예측정확도를 확인
print(f"# Train set score: {NB_clf.score(X_train_tfidf, y_train):.3f}")
# test set에 대한 예측정확도를 확인
print(f"# Test set score: {NB_clf.score(X_test_tfidf, y_test):.3f}")

# Train set dimension: (2034, 20085)
# Test set dimension: (1353, 20085)
# Train set score: 0.968
# Test set score: 0.768
# Train set score: 0.971
# Test set score: 0.793


위 예에서는 로지스틱 회귀분석의 결과를 향상시키기 위해 그리드서치로 적정 alpha 값 2.4를 구한 후, 릿지 회귀분석을 실시했다. 그리고, 지금까지 수행했던 문서 분류 알고리즘 중에서 나이브 베이즈의 결과가 0.741로 가장 좋았던 것을 상기하고 나이브 베이즈 분류도 수행했다. 결과를 보면, 릿지 회귀분석도 0.768로 꽤 향상됐지만 나이브 베이즈 분류도 수행했다. 결과를 보면, 릿지 회귀분석도 0.768로 꽤 향상됐지만 나이브 베이즈는 0.793으로 지금까지의 결과 중에서 가장 높은 결과를 보여준다. 이보다 좋은 결과를 얻기 위해 다양한 시도를 해보길 추천한다.